# Laboratorium 6 - rekomendacje oparte na grafach wiedzy

## Przygotowanie

 * pobierz i wypakuj dataset: https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge
   * na potrzeby drugiej części laboratorium (czyli testowego treningu), na Teamsach macie dostępny podzbiór danych, `a_few_playlists_dataset` - nie wystarczy on jednak do wykonania trzeciej części (i tym samym do oddania laboratorium)
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab6`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas pykeen tqdm seaborn`

## Część 1. - przygotowanie danych

In [5]:
import json
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import random

from pykeen.models import TransE
from pykeen.pipeline import pipeline
from pykeen.predict import predict_target
from pykeen.triples import TriplesFactory

/home/michal/Rekomendacyjne/recsyslab1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # konfiguracja datasetu Spotify
# PATH = 'spotify_million_playlist_dataset/data'
# SAMPLING_RATIO = 1.0
# FILENAMES = random.sample([f'mpd.slice.{1000*i}-{1000*i+999}.json' for i in range(1000)], int(SAMPLING_RATIO*1000))

In [6]:
# jesli uzywasz datasetu pobranego z Teamsow, uzyj tej komorki zamiast powyzszej
# UWAGA - do oddania laboratorium konieczne jest uzycie oficjalnego datasetu Spotify
PATH = 'a_few_playlists_dataset'
SAMPLING_RATIO = 0.01
with open(f'{PATH}/filenames.txt') as fn:
    FILENAMES = fn.read().split('\n')

In [7]:
# funkcje do parsowania playlist
def get_id(uri):
    return uri.split(':')[-1]

def parse_playlist(playlist):
    name = playlist['name']
    tracks = [get_id(t['track_uri']) for t in playlist['tracks']]
    tracks_to_artists = {(get_id(t['track_uri']), get_id(t['artist_uri'])) for t in playlist['tracks']}
    tracks_to_albums = {(get_id(t['track_uri']), get_id(t['album_uri'])) for t in playlist['tracks']}
    albums_to_artists = {(get_id(t['album_uri']), get_id(t['artist_uri'])) for t in playlist['tracks']}
    return name, tracks, tracks_to_artists, tracks_to_albums, albums_to_artists

In [8]:
# parsing
playlists = []
tracks = set()
tracks_to_artists = set()
tracks_to_albums = set()
albums_to_artists = set()

for filename in tqdm(FILENAMES):
    with open(f'{PATH}/{filename}') as mpd_chunk:
        for playlist in json.loads(mpd_chunk.read())['playlists']:
            a, b, c, d, e = parse_playlist(playlist)
            playlists.append(b)
            tracks.update(set(b))
            tracks_to_artists.update(c)
            tracks_to_albums.update(d)
            albums_to_artists.update(e)

print(f'Got: {len(playlists):,} playlists; {len(tracks):,} tracks; {len({x[1] for x in tracks_to_albums}):,} albums; {len({x[1] for x in tracks_to_artists}):,} artists')

100%|██████████| 10/10 [00:11<00:00,  1.17s/it]


Got: 10,000 playlists; 175,992 tracks; 84,206 albums; 36,844 artists


In [9]:
# w zbiorze testowym chcemy tylko te playlisty, ktorych wszystkie piosenki wystapia takze choc raz w zbiorze treningowym
tracks_counter = {}
for p in tqdm(playlists):
    for t in p:
        if t in tracks_counter:
            tracks_counter[t] += 1
        else:
            tracks_counter[t] = 1

playlists_ids_with_only_non_unique_tracks = []
for i in tqdm(range(len((playlists)))):
    p = playlists[i]
    if all([tracks_counter[t] > 1 for t in p]):
        playlists_ids_with_only_non_unique_tracks.append(i)

# zbior testowy to 1/10 wszystkich playlist - czyli 100k, jesli nie używamy samplingu
test_playlist_ids = random.sample(playlists_ids_with_only_non_unique_tracks, int(SAMPLING_RATIO*100_000))
# zbior treningowy to cala reszta playlist - jest ich duzo, wiec sprobujmy to zrobic wydajnie
test_ids_sorted = sorted(test_playlist_ids)
test_i = 0
train_playlist_ids = []
i = 0
while i < len(playlists):
    if test_i < len(test_ids_sorted) and test_ids_sorted[test_i] == i:
        test_i += 1
    else:
        train_playlist_ids.append(i)
    i += 1

train_playlists = [playlists[i] for i in train_playlist_ids]
test_playlists = [playlists[i] for i in test_playlist_ids]

print(f'train dataset: {len(train_playlists):,}; test dataset: {len(test_playlists):,}')

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:00<00:00, 24903.81it/s]


train dataset: 9,000; test dataset: 1,000


## Część 2. - budowa i ewaluacja modelu

### Relacje istniejące w naszym datasecie:
![poglądowy obrazek relacji w datasecie](relations.svg)

In [10]:
# budowanie zbioru relacji
# mozesz usunac czesc wpisow z listy `relations`
relations = ['follows', 'authored_by', 'in_album', 'authored']
triples = []

# relacje piosenka -> autor
if 'authored_by' in relations:
    for track, artist in tqdm(tracks_to_artists):
        triples.append((track, 'authored_by', artist))

# relacje piosenka -> artysta
if 'in_album' in relations:
    for track, album in tqdm(tracks_to_albums):
        triples.append((track, 'in_album', album))

# relacje artysta -> album
if 'authored' in relations:
    for album, artist in tqdm(albums_to_artists):
        triples.append((artist, 'authored', album))

# relacje piosenka -> piosenka
if 'follows' in relations:
    for playlist in tqdm(train_playlists):
        for i in range(len(playlist)-1):
            triples.append((playlist[i], 'follows', playlist[i+1]))

num_entities = len(triples)
num_relations = len(relations)

print(f'Got {num_relations} relations with total of {num_entities:,} entries')

  0%|          | 0/175992 [00:00<?, ?it/s]

100%|██████████| 9000/9000 [00:00<00:00, 17588.40it/s]

Got 4 relations with total of 1,065,707 entries


In [11]:
# trening
tf = TriplesFactory.from_labeled_triples(np.array(triples))
# ta dysproporcja jest po to, by szybko uzyskac jakikolwiek wynik
#   - dla uzyskania sensownych wynikow warto zmienic split na np. standardowe 80-10-10
training, testing, validation = tf.split([.8, .1, .1])

pipeline_result = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model=TransE, # to najszybszy i najprostszy, ale i najgorszy model; pomysl o uzyciu TransH, TransR, RESCAL albo dowolnego innego
    epochs=1 # to zdecydowanie za malo - 1 wystarczy do jakichkolwiek wynikow, 5 do dosc slabych, blizej 20 do sensownych
)

using automatically assigned random_state=2027330857
No random seed is specified. Setting to 1473018507.
No cuda devices were available. The model runs on CPU
Training epochs on cpu: 100%|██████████| 1/1 [22:14<00:00, 1334.17s/epoch, loss=1.03, prev_loss=nan]
INFO:pykeen.evaluation.evaluator:Currently automatic memory optimization only supports GPUs, but you're using a CPU. Therefore, the batch_size will be set to the default value.
INFO:pykeen.evaluation.evaluator:No evaluation batch_size provided. Setting batch_size to '32'.
Evaluating on cpu:   2%|▏         | 2.11k/102k [13:42<10:24:28, 2.65triple/s]

In [ ]:
# zgrubne oszacowanie jakosci wytrenowanego modelu
pipeline_result.plot_losses()

In [1]:
# funkcja zwracajaca predykcje z modelu
def predict_next_tracks(track_id: str, k: int) -> list[str]:
    return predict_target(
        model=pipeline_result.model, 
        triples_factory=tf,
        relation='follows',
        head=track_id,
    ).df.head(k)

In [ ]:
# metryki do porownania rekomenderow - precision@k i recall@k
def precision(prediction: list[str], actual_tracks: list[str]) -> float:
    return len(set(prediction) & set(actual_tracks)) / len(prediction)

def recall(prediction: list[str], actual_tracks: list[str]) -> float:
    return len(set(prediction) & set(actual_tracks)) / len(actual_tracks)

In [ ]:
# ocena wynikow

# odsiewamy playlisty zbyt krotkie, by dac sensowne wyniki
long_enough_test_playlists = [p for p in test_playlists if len(p) >= 10]
# z kazdej playlisty, elementy od 0 do `cutoff_idx` wlacznie sa dane, na ich podstawie robimy predykcje
# elementy od cutoff_idx+1 do konca powinnismy umiec przewidziec
cutoff_idx = 4
# ile elementow ma przewidziec nasz model
k = 20
precisions: list[float] = []
recalls: list[float] = []

for playlist in tqdm(long_enough_test_playlists):
    prediction = predict_next_tracks(playlist[cutoff_idx], k)
    precisions.append(precision(prediction, playlist[cutoff_idx+1:]))
    recalls.append(recall(prediction, playlist[cutoff_idx+1:]))

# histogram z wynikami
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.hist(precisions, bins=20)
ax1.set_title('Precision')
ax2.hist(recalls, bins=20)
ax2.set_title('Recall')
plt.show()

## Część 3. - porównanie różnych metod rekomendacji

W części 2. zbudowaliśmy zbiór trójek relacji, wytrenowaliśmy jeden model, zaimplementowaliśmy (prostą) metodę generującą rekomendacje na podstawie predykcji modelu i w końcu zaimplementowaliśmy dwie metryki do porównania jakości tych rekomendacji.

W części 3. Twoim zadaniem jest przetestować trzy różne podejścia do jednego z kroków:
1. Porównaj trzy różne modele spośród dostępnych w bibliotece PyKeen: https://pykeen.readthedocs.io/en/stable/reference/models.html#classes
   * jeden model translacyjny (np. TransE, TransH, TransR)
   * jeden model faktoryzacyjny (np. RESCAL)
   * jeden dowolny model niewybrany w poprzednich punktach
2. Porównaj trzy metody budowania grafu wiedzy:
   * graf zawierający relacje wszystkich czterech typów
   * graf zawierający tylko relacje typu `follows` (czyli między kolejnymi utworami w playliście)
   * graf zawierający relacje wybranych przez Ciebie dwóch lub trzech typów (czyli krok pośredni między powyższymi punktami)
3. Porównaj trzy metody generowania rekomendacji na podstawie elementów zwróconych przez `predict_target()` (ta metoda zwraca m. in. score'y każdego z proponowanych elementów, co może okazać się pomocne):
   * metoda opierająca się tylko na predykcji dla ostatniego znanego elementu w playliście
   * dwie wymyślone przez Ciebie, bardziej zaawansowane metody
  
Niezależnie od tego, który z trzech powyższych scenariuszy wybierzesz - porównaj trzy wybrane przez Ciebie metody na podstawie histogramów metryk `precision@k` i `recall@k`.

In [ ]:
options = [['follows'], ['authored_by', 'in_album'], ['follows', 'authored_by', 'in_album', 'authored']]

for option in options:
    triples = []
    if 'authored_by' in option:
        for track, artist in tqdm(tracks_to_artists):
            triples.append((track, 'authored_by', artist))
    if 'in_album' in option:
        for track, album in tqdm(tracks_to_albums):
            triples.append((track, 'in_album', album))
    if 'authored' in option:
        for album, artist in tqdm(albums_to_artists):
            triples.append((artist, 'authored', album))
    if 'follows' in option:
        for playlist in tqdm(train_playlists):
            for i in range(len(playlist)-1):
                triples.append((playlist[i], 'follows', playlist[i+1]))
    tf = TriplesFactory.from_labeled_triples(np.array(triples))
    training, testing, validation = tf.split([.8, .1, .1])
    pipeline_result = pipeline(
        training=training,
        testing=testing,
        validation=validation,
        model=TransE,
        epochs=15
    )
    pipeline_result.plot_losses()
    cutoff_idx = 4
    k = 20
    precisions: list[float] = []
    recalls: list[float] = []

    def predict_next_tracks(track_id: str, k: int) -> list[str]:
        return predict_target(
            model=pipeline_result.model,
            query=track_id,
            k=k,
        )

    for playlist in tqdm(long_enough_test_playlists):
        prediction = predict_next_tracks(playlist[cutoff_idx], k)
        precisions.append(precision(prediction, playlist[cutoff_idx+1:]))
        recalls.append(recall(prediction, playlist[cutoff_idx+1:]))
    
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.hist(precisions, bins=20)
    ax1.set_title('Precision')
    ax2.hist(recalls, bins=20)
    ax2.set_title('Recall')
    ax2.set_title('Recall')
    plt.show()